# Texas Tow Trucks (`.apply` and `requests`)

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
import requests

from selenium import webdriver

## Search for the TLDR Number `006565540C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')

In [3]:
button = driver.find_element_by_xpath('/html/body/div/div[1]/div/div/form/table/tbody/tr[5]/td[1]/input[1]')
button.click()

In [4]:
textbox = driver.find_element_by_xpath('/html/body/div/div[1]/div/div/form/table/tbody/tr[5]/td[1]/input[2]')
textbox.send_keys("006565540C")

In [5]:
submit_button = driver.find_element_by_xpath('/html/body/div/div[1]/div/div/form/table/tbody/tr[10]/td/center/button')
submit_button.click()

In [6]:
company_table = driver.find_elements_by_tag_name('table')[4]
name = company_table.find_elements_by_tag_name('td')[2].text
owner = company_table.find_elements_by_tag_name('td')[4].text
phone = company_table.find_elements_by_tag_name('td')[6].text
company_table_2 = driver.find_elements_by_tag_name('table')[5]
status = company_table_2.find_elements_by_tag_name('td')[1].text
address = company_table_2.find_elements_by_tag_name('td')[3]
addres = address.text.split('\n')
address_correct = addres[8] + addres[9] + addres [10]

print(name, owner, phone, status, address_correct)

Name:    H & A TOWING LLC Owner/Officer:   HANEEN ABBAS MOHAMMEDAWI / MANAGER Phone:   512-999-8883 Status:  Active Physical:11710 JOSEPH CLAYTON DRAUSTIN, TX. 78753


# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [7]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
df = pd.read_csv("trucks-subset.csv")
df.head()

/Users/biancapallaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,TDLR Number
0,006565540C
1,0654479VSF
2,006564940C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C`.

- *TIP: Use .apply and a function*
- *TIP: You'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', None)` to display aaaalll of the text in a cell*

In [8]:
def url(cell):
    print("https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + cell)


In [9]:
df.apply(url)

0    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C
1    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0654479VSF
2    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006564940C
Name: TDLR Number, dtype: object


TDLR Number    None
dtype: object

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [22]:
def url(cell):
    dataframe_column = "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + cell
    return dataframe_column

In [27]:
df['url'] = df.apply(url)
df.head()

,TDLR Number,url
0,006565540C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C
1,0654479VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0654479VSF
2,006564940C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006564940C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [28]:
def scrape(url):
    try:
        driver.get(url)
        driver.get(url)
        company_table = driver.find_elements_by_tag_name('table')[4]
        name = company_table.find_elements_by_tag_name('td')[2].text
        owner = company_table.find_elements_by_tag_name('td')[4].text
        phone = company_table.find_elements_by_tag_name('td')[6].text
        company_table_2 = driver.find_elements_by_tag_name('table')[5]
        status = company_table_2.find_elements_by_tag_name('td')[1].text
        address = company_table_2.find_elements_by_tag_name('td')[3]
        addres = address.text.split('\n')
        address_correct = addres[8] + addres[9] + addres [10]
        return (name, owner, phone, status, address_correct)
    except:
        pass

In [29]:
df.url.apply(scrape)

0                                (Name:    H & A TOWING LLC, Owner/Officer:   HANEEN ABBAS MOHAMMEDAWI / MANAGER, Phone:   512-999-8883, Status:  Active, Physical:11710 JOSEPH CLAYTON DRAUSTIN, TX. 78753)
1    (Name:    24/7 TOWING AND RECOVERY LLC, Owner/Officer:   SCOTT JENSON / VICE PRESIDENT, Owner/Officer:   CHRIS BRENES / PRESIDENT, Status:  Active, Physical:3601 N COUNTY ROAD 1148MIDLAND, TX. 79705)
2                                     (Name:    A & N TOWING LLC, Owner/Officer:   ABDULRAHMAN Z ALABDULLAH / CEO, Phone:   2106678546, Status:  Active, Physical:10838 DEEPWATER BAYSAN ANTONIO, TX. 78251)
Name: url, dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [51]:
def scrape(url):
    try:
        driver.get(url)
        driver.get(url)
        company_table = driver.find_elements_by_tag_name('table')[4]
        name = company_table.find_elements_by_tag_name('td')[2].text
        owner = company_table.find_elements_by_tag_name('td')[4].text
        phone = company_table.find_elements_by_tag_name('td')[6].text
        company_table_2 = driver.find_elements_by_tag_name('table')[5]
        status = company_table_2.find_elements_by_tag_name('td')[1].text
        address = company_table_2.find_elements_by_tag_name('td')[3]
        addres = address.text.split('\n')
        address_correct = addres[8] + addres[9] + addres [10]
        data = {
            'Name': name,
            'Owner': owner,
            'Phone': phone,
            'Status': status,
            'Address': address_correct     
        }
        return pd.Series(data)
        
    except:
        pass

In [52]:
df2 = df.url.apply(scrape)

In [53]:
df2

,Name,Owner,Phone,Status,Address
0,Name: H & A TOWING LLC,Owner/Officer: HANEEN ABBAS MOHAMMEDAWI / MANAGER,Phone: 512-999-8883,Status: Active,"Physical:11710 JOSEPH CLAYTON DRAUSTIN, TX. 78753"
1,Name: 24/7 TOWING AND RECOVERY LLC,Owner/Officer: SCOTT JENSON / VICE PRESIDENT,Owner/Officer: CHRIS BRENES / PRESIDENT,Status: Active,"Physical:3601 N COUNTY ROAD 1148MIDLAND, TX. 79705"
2,Name: A & N TOWING LLC,Owner/Officer: ABDULRAHMAN Z ALABDULLAH / CEO,Phone: 2106678546,Status: Active,"Physical:10838 DEEPWATER BAYSAN ANTONIO, TX. 78251"


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [54]:
df.to_csv(r'/Users/biancapallaro/Documents/Foundations/tow_trucks_extended.csv', index = False, header=True)

### Re-open your dataframe to confirm you didn't save any extra weird columns

## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**